# Crime Analysis

In this notebook we first preprocess the dataset of crimes. <br/>


First we import libraries needed.

In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
import nltk

After importing needed libraries, we load the data.

In [3]:
# Load the dataset
data = pd.read_csv('crime.csv', encoding='iso-8859-1')

/tmp/ipykernel_1897449/3746609399.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('crime.csv', encoding='iso-8859-1')


After that it's time for data reduction for `OFFENSE_CODE` and `OFFENSE_DESCRIPTION`. <br/>
because these two columns are so important, if in any row we have lost data of them, it's better to remove the whole column.

In [4]:
# Data reduction
imputer_c = SimpleImputer(strategy='constant')
data.drop(index=data['OFFENSE_CODE'].isnull().index, axis=1)
data.drop(index=(data['OFFENSE_DESCRIPTION'] == '').index, axis=1)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location


After that we handle the missing values.

In [5]:
# Handle missing values
imputer = SimpleImputer(strategy='mean')
data['Lat'] = imputer.fit_transform(data[['Lat']])
data['Long'] = imputer.fit_transform(data[['Long']])

data['REPORTING_AREA'].fillna(data['REPORTING_AREA'].mode().iloc[0])
data['SHOOTING'].fillna('N', inplace=True)
data['OCCURRED_ON_DATE'].fillna(data['OCCURRED_ON_DATE'].mode().iloc[0])
data['YEAR'].fillna(data['YEAR'].mode().iloc[0])
data['MONTH'].fillna(data['MONTH'].mode().iloc[0])
data['DAY_OF_WEEK'].fillna(data['DAY_OF_WEEK'].mode().iloc[0])
data['HOUR'].fillna(data['HOUR'].mode().iloc[0])
data['UCR_PART'].fillna(data['UCR_PART'].mode().iloc[0])
data['STREET'].fillna(data['STREET'].mode().iloc[0])

0              ARLINGTON ST
1                ALLSTON ST
2                  DEVON ST
3              CAMBRIDGE ST
4               PRESCOTT ST
                ...        
327815          COVENTRY ST
327816             RIVER ST
327817             RIVER ST
327818    NEW WASHINGTON ST
327819        WASHINGTON ST
Name: STREET, Length: 327820, dtype: object